# Сбор данных с помощью VK API

* [VK API](https://dev.vk.com/ru/method/)
* [Статья на хабре](https://habr.com/ru/articles/720862/)

Из прочитанного: подойдет сбор данных из новостной ленты по запросу (`newsfeed.search` с параметром `q`)


Вывод записей ограничен, значит [нужно выгружать данные в цикле, постранично](https://dev.vk.com/ru/reference/version/5.13#:~:text=next_from%20%E2%80%94%20%D0%B7%D0%BD%D0%B0%D1%87%D0%B5%D0%BD%D0%B8%D0%B5%2C%20%D0%BA%D0%BE%D1%82%D0%BE%D1%80%D0%BE%D0%B5%20%D0%BF%D1%80%D0%B8%D1%85%D0%BE%D0%B4%D0%B8%D1%82%20%D0%B2,%D0%BA%D0%BE%D1%82%D0%BE%D1%80%D0%BE%D0%B5%20%D0%BE%D0%B1%D0%BE%D0%B7%D0%BD%D0%B0%D1%87%D0%B0%D0%B5%D1%82%20%D0%BD%D0%B0%D1%87%D0%B0%D0%BB%D0%BE%20%D1%81%D0%BB%D0%B5%D0%B4%D1%83%D1%8E%D1%89%D0%B5%D0%B3%D0%BE%20%D0%B4%D0%B8%D0%B0%D0%BF%D0%B0%D0%B7%D0%BE%D0%BD%D0%B0.)

In [6]:
import requests
from datetime import datetime

VK_API = 'https://api.vk.com/method/newsfeed.search'
VK_API_VERSION = '5.199'
ACCESS_TOKEN = 'YOUR TOKEN'
TAG = 'СПБГУ'
TAG2 = 'МГУ'

tmp_datetime = datetime(2024, 8, 1, 15, 0, 0)
end_datetime = datetime(2024, 8, 1, 16, 0, 0)
print(tmp_datetime, end_datetime)
print(tmp_datetime.timestamp(), end_datetime.timestamp())

params = {
            'access_token': ACCESS_TOKEN,
            'count': 200,
            'q': TAG,
            'v': VK_API_VERSION,
            "start_time": tmp_datetime.timestamp(),
            "end_time": end_datetime.timestamp()
        }

2024-08-01 15:00:00 2024-08-01 16:00:00
1722513600.0 1722517200.0


In [31]:
resp = requests.get(VK_API, params)

In [ ]:
if 'error' in resp.json().keys(): print('fkfk')

In [2]:
params['start_from'] = resp.json()['response']['next_from']

In [3]:
resp = requests.get(VK_API, params)

In [ ]:
resp.json()['response']['next_from']

In [8]:
data = resp.json()['response']['items']
len(data), data

(100,
 [{'inner_type': 'wall_wallpost',
   'donut': {'is_donut': False},
   'comments': {'can_post': 1,
    'can_view': 1,
    'count': 0,
    'groups_can_post': True},
   'marked_as_ads': 0,
   'type': 'post',
   'carousel_offset': 0,
   'attachments': [{'type': 'photo',
     'photo': {'album_id': -7,
      'date': 1726611417,
      'id': 457246535,
      'owner_id': 9615067,
      'access_key': '5750c089c49a49a0cb',
      'sizes': [{'height': 59,
        'type': 's',
        'width': 75,
        'url': 'https://sun1-99.userapi.com/s/v1/ig2/tHU17k0kDmM94hoxk21bzePR1nsHAb4G3GWJPhjPtTQetOIcJhoC3WdCFz6Er4u4SDXck1vUqXeA-_k-iWTCmfJE.jpg?quality=95&as=32x25,48x38,72x57,108x85,160x126,240x188,360x283,480x377,540x424,640x502,720x565,1080x848,1280x1005,1440x1131,1536x1206&from=bu&cs=75x59'},
       {'height': 102,
        'type': 'm',
        'width': 130,
        'url': 'https://sun1-99.userapi.com/s/v1/ig2/tHU17k0kDmM94hoxk21bzePR1nsHAb4G3GWJPhjPtTQetOIcJhoC3WdCFz6Er4u4SDXck1vUqXeA-_k-iWTCmf

In [9]:
data[0].keys()

dict_keys(['inner_type', 'donut', 'comments', 'marked_as_ads', 'type', 'carousel_offset', 'attachments', 'date', 'from_id', 'id', 'likes', 'reaction_set_id', 'owner_id', 'post_source', 'post_type', 'reposts', 'text', 'views'])

In [18]:
data[0]['reposts']

{'count': 0, 'user_reposted': 0}

### Функция для сбора данных за период

In [7]:
from datetime import datetime, timedelta
import pandas as pd
import time

def parse_vk_posts_info(data):
    '''
    Извлечь из выгруженных данных ценную информацию
    
    Parameters
    ----------
    data : list
        массив словарей, которые содержат данные о публикациях
    '''
    result = {
        "id": [],
        "from_id": [],
        "date": [],
        "text": [],
        "likes": [],
        "views": [],
        "reposts": [],
        "comments": [],               
    }
    for record in data:
        if record['post_type'] == 'post':
            result['id'].append(record['id'])
            result['date'].append(datetime.fromtimestamp(record['date']).strftime('%Y-%m-%d'))
            result['from_id'].append(record['from_id'])
            result['text'].append(record['text'])
            result['likes'].append(record.get('likes', {'count': 0})['count'])
            result['views'].append(record.get('views', {'count': 0})['count'])
            result['reposts'].append(record.get('reposts', {'count': 0})['count'])
            result['comments'].append(record.get('comments', {'count': 0})['count'])
        else:
            continue

    return pd.DataFrame(result)


In [8]:
df = parse_vk_posts_info(resp.json()['response']['items'])
df.head()

NameError: name 'resp' is not defined

In [9]:
def get_posts_from_vkapi(start_datetime, end_datetime, topic):
    '''Извлечь данные из vk-api'''
    
    assert isinstance(start_datetime, datetime), 'Неверный формат данных'
    assert isinstance(end_datetime, datetime), 'Неверный формат данных'

    tmp_datetime = start_datetime
    delta = timedelta(hours=1)
    result = pd.DataFrame(columns=['id',
                                   'from_id',
                                   'date',
                                   'text',
                                   'views',
                                   'reposts',
                                   'likes',
                                   'comments'])
    
    # цикл по времени
    while tmp_datetime <= end_datetime:
        parameters = {
            "access_token": ACCESS_TOKEN,
            'count': 200,
            "q": topic,
            "v": VK_API_VERSION,
            "start_time": tmp_datetime.timestamp(),
            "end_time": end_datetime.timestamp()
        }
        # вызов метода API
        response = requests.get(VK_API, parameters)
        
        if 'error' in response.json():
            print('Неудачный запрос по API')
            return None
        else:   
            parsed_data = parse_vk_posts_info(response.json()['response']['items'])
            result = pd.concat([parsed_data, result])
        
        tmp_datetime += delta
        # чтобы не было connection broken
        time.sleep(0.3)
    
    return result

In [10]:
df = get_posts_from_vkapi(start_datetime=datetime(2024, 9, 1),
                          end_datetime=datetime(2024, 9, 2),
                          topic='МГУ')

In [11]:
df.head()

,id,from_id,date,text,likes,views,reposts,comments
0,37430,396915837,2024-09-01,2 сентября 80 лет назад родился Александр Фили...,1,219,0,0
1,49649,737716395,2024-09-01,Приметы 1 сентября: что нельзя делать в День з...,2,79,0,0
2,2179,-161405799,2024-09-01,"Минюст РФ признал иноагентом Елену Лукьянову, ...",1,22,1,0
3,1464,-92972603,2024-09-01,День знаний – 2024 в МГУ! \n\n2 сентября пройд...,2,150,1,0
4,100,212948872,2024-09-01,"Лето своей мечты, как мечталось, так и сбыли? ...",21,184,0,0
